<a href="https://colab.research.google.com/github/YakovDeLeon/ML_Course_PT/blob/main/Lecture4_Clustering%26MLSD/PT_Practice4_WhoIsTalking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Занятие 4. Who’s Talking? – Classify Encrypted TCP Traffic

Сегодня вам предстоит работать с реальным зашифрованным корпоративным сетевым трафиком.
Ваша задача — классифицировать, какое приложение или сервис сгенерировал данный TCP-поток, **несмотря на то, что полезная нагрузка зашифрована**, и доступны только размеры пакетов и их направления.

Датасет был собран в контролируемой корпоративной среде, где каждой сетевой сессии было присвоено соответствующее приложение или сервис.
Каждая запись представляет собой усечённый TCP-поток длиной до 30 пакетов и содержит только информацию о длине пакетов и направлении передачи.
Такой формат позволяет исследовать, как методы машинного обучения могут выявлять поведенческие паттерны в зашифрованном трафике без анализа его содержимого.

### Описание данных

Данные содержат размеченные примеры TCP-потоков с соответствующим приложением или сервисом (`app_service`).

**Колонки:**

* **app_service** — целевая переменная: название или числовой идентификатор приложения или сервиса, сгенерировавшего TCP-поток (например, Telegram, YouTube, Zoom)

* **tcp_len_1 – tcp_len_30** — последовательность длин до 30 TCP-пакетов внутри потока.
- Положительные значения соответствуют пакетам, отправленным клиентом.
- Отрицательные значения соответствуют пакетам, полученным от сервера.
-  Если сессия содержит меньше 30 пакетов, оставшиеся позиции заполняются нулями.

# <font color="green">Осознание целей и ограничений бизнеса - вопросы для обсуждения с заказчиком</font>

### Бизнес и цель

- Для чего нужна классификация трафика?

Примеры: мониторинг использования приложений, защита корпоративной сети, аудит использования SaaS, оптимизация пропускной способности.

- Какие приложения/сервисы нас интересуют? Только конкретные ключевые (Zoom, YouTube, Telegram), или все возможные?

- Какое требование к точности классификации? Например, достаточно 80% или нужно >95%?


### Данные и сбор

- Насколько репрезентативен датасет по сравнению с реальностью?

Примеры: различное время суток, разные сети (Wi-Fi, проводная), разные пользователи.

- Как часто появляются новые приложения или обновления существующих? Как быть с классами «новых» приложений?

- Есть ли ограничения на доступ к метаданным трафика, кроме длины пакетов и направления?


### Ограничения на модель

- Время отклика: нужен ли реальный онлайн детектор для каждого потока или можно пакетно/батчево?

- Ресурсы: допустимая нагрузка на CPU/GPU, память, скорость обработки.

- Частота обновления модели: сколько раз в неделю/месяц будем переобучать?


### Конфиденциальность и безопасность

- Данные трафика зашифрованы. Нужно ли дополнительно анонимизировать метаданные или учитывать регуляторные ограничения?

# <font color="green">Особенности данных и подготовка </font>

### Формат данных

- Последовательности TCP-пакетов длиной до 30, с положительными/отрицательными значениями.

- Пустые позиции заполнены нулями.

- Целевая переменная — app_service.


### Возможные pre-processing шаги

- Нормализация длин пакетов (например, деление на 1500 для приведения к [−1,1]).


### Дополнительные признаки

- Статистика по пакету: средняя длина, медиана, стандартное отклонение.

- Количество пакетов в каждом направлении (up/down).

- Интервалы между пакетами (если доступны временные метки).

- Можно использовать sequence models (RNN, LSTM, Transformer) или feature-based models (Random Forest, XGBoost).


### Особенности

- Потоки разной длины → padding до 30.

- Потоки сильно разрежены → много нулей.

- Много приложений → возможно сильный class imbalance.

# <font color="green">Бизнес и офлайн-метрики </font>

### Бизнес-метрики

- Доля правильно классифицированных приложений в онлайн-сценарии.

- Доля ошибочной классификации для критичных приложений (Zoom, корпоративные SaaS).

- Влияние на решения по безопасности и мониторингу (например, блокировка нежелательных сервисов).


### Офлайн ML-метрики

- Accuracy / Macro F1 / Weighted F1 (особенно при несбалансированных классах)

- Confusion matrix → какие приложения путаются чаще всего

- Top-K accuracy (например, Top-3, если нужно знать несколько вероятных кандидатов)

- ROC-AUC для каждого класса (multi-class AUC)

- Precision/Recall для ключевых приложений (например, высокочастотных или критичных)


### Особые моменты для офлайн-метрик

- Обратить внимание на имитацию реальной среды: train/test split должен быть по сессиям и пользователям, чтобы модель не запоминала конкретные шаблоны отдельных пользователей.

Сначала скачаем данные

In [5]:
!wget -O response.json \
"https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/7LCvNsGL1mv90Q"

--2026-02-28 21:27:54--  https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/7LCvNsGL1mv90Q
Resolving cloud-api.yandex.net (cloud-api.yandex.net)... 213.180.204.127, 2a02:6b8::1:127
Connecting to cloud-api.yandex.net (cloud-api.yandex.net)|213.180.204.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 617 [application/json]
Saving to: ‘response.json’

response.json       100%[===================>]     617  --.-KB/s    in 0s      

2026-02-28 21:27:55 (42.8 MB/s) - ‘response.json’ saved [617/617]



In [6]:
import json

with open("response.json") as f:
    data = json.load(f)

print(data["href"])

https://downloader.disk.yandex.ru/disk/a5788fff2e6ae8f4417d313a048440f401e4528e6eb5e3168e0a10cccf3f8a61/69a3961b/fKqInKw3d7bLFOeFnMGnhOEt74e_AcHrfoeOiWQFqnxSx2KlJFe3moqi-rTFCO6djYu3q6EHAUP4X1BQu7yk7eyLDbl9lafVd4ruaDAPlxGr8npumZHI4midPdWhecNq?uid=0&filename=whos-talking-classify-the-app-by-its-packets.zip&disposition=attachment&hash=m8VLCd1u1YCGUik9dZVkJjbxXRNbrE0kS1p%2BEIBguw1klnm56L9GDaExZ6Yk0MeDq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=259847718&fsize=305374529&hid=11ebcd4213988e57481af625f75bf49e&media_type=compressed&tknv=v3


In [7]:
!wget -O archive.zip "https://downloader.disk.yandex.ru/disk/5dde852b493639fd57bcbd6ed949c9d12dcfe80dbbf647f577a410b288bb282e/69a395e0/fKqInKw3d7bLFOeFnMGnhOEt74e_AcHrfoeOiWQFqnxSx2KlJFe3moqi-rTFCO6djYu3q6EHAUP4X1BQu7yk7eyLDbl9lafVd4ruaDAPlxGr8npumZHI4midPdWhecNq?uid=0&filename=whos-talking-classify-the-app-by-its-packets.zip&disposition=attachment&hash=m8VLCd1u1YCGUik9dZVkJjbxXRNbrE0kS1p%2BEIBguw1klnm56L9GDaExZ6Yk0MeDq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=259847718&fsize=305374529&hid=11ebcd4213988e57481af625f75bf49e&media_type=compressed&tknv=v3"
# !wget -O archive.zip str(data["href"])

--2026-02-28 21:27:55--  https://downloader.disk.yandex.ru/disk/5dde852b493639fd57bcbd6ed949c9d12dcfe80dbbf647f577a410b288bb282e/69a395e0/fKqInKw3d7bLFOeFnMGnhOEt74e_AcHrfoeOiWQFqnxSx2KlJFe3moqi-rTFCO6djYu3q6EHAUP4X1BQu7yk7eyLDbl9lafVd4ruaDAPlxGr8npumZHI4midPdWhecNq?uid=0&filename=whos-talking-classify-the-app-by-its-packets.zip&disposition=attachment&hash=m8VLCd1u1YCGUik9dZVkJjbxXRNbrE0kS1p%2BEIBguw1klnm56L9GDaExZ6Yk0MeDq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=259847718&fsize=305374529&hid=11ebcd4213988e57481af625f75bf49e&media_type=compressed&tknv=v3
Resolving downloader.disk.yandex.ru (downloader.disk.yandex.ru)... 77.88.21.127, 2a02:6b8::2:127
Connecting to downloader.disk.yandex.ru (downloader.disk.yandex.ru)|77.88.21.127|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s494vla.storage.yandex.net/rdisk/5dde852b493639fd57bcbd6ed949c9d12dcfe80dbbf647f577a410b288bb282e/69a395e0/fKqInKw3d7bLFOeFnMGnhOEt74e_A

In [4]:
!unzip archive.zip -d data

Archive:  archive.zip
replace data/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Установим необходимые библиотеки

In [8]:
!pip install catboost -q

import numpy as np
import pandas as pd
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool

Загрузим данные и посмотрим на них

In [9]:
df = pd.read_csv('/content/data/train.csv')

df.info()

/tmp/ipython-input-33074/525312853.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/data/train.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8248546 entries, 0 to 8248545
Data columns (total 31 columns):
 #   Column       Dtype  
---  ------       -----  
 0   app_service  object 
 1   tcp_len_1    int64  
 2   tcp_len_2    float64
 3   tcp_len_3    float64
 4   tcp_len_4    float64
 5   tcp_len_5    float64
 6   tcp_len_6    float64
 7   tcp_len_7    float64
 8   tcp_len_8    float64
 9   tcp_len_9    float64
 10  tcp_len_10   float64
 11  tcp_len_11   float64
 12  tcp_len_12   float64
 13  tcp_len_13   float64
 14  tcp_len_14   float64
 15  tcp_len_15   float64
 16  tcp_len_16   float64
 17  tcp_len_17   float64
 18  tcp_len_18   float64
 19  tcp_len_19   float64
 20  tcp_len_20   float64
 21  tcp_len_21   float64
 22  tcp_len_22   float64
 23  tcp_len_23   float64
 24  tcp_len_24   float64
 25  tcp_len_25   float64
 26  tcp_len_26   float64
 27  tcp_len_27   float64
 28  tcp_len_28   float64
 29  tcp_len_29   float64
 30  tcp_len_30   float64
dtypes: float64(29), int64(1)

In [10]:
df.sample(5)

,app_service,tcp_len_1,tcp_len_2,tcp_len_3,tcp_len_4,tcp_len_5,tcp_len_6,tcp_len_7,tcp_len_8,tcp_len_9,...,tcp_len_21,tcp_len_22,tcp_len_23,tcp_len_24,tcp_len_25,tcp_len_26,tcp_len_27,tcp_len_28,tcp_len_29,tcp_len_30
7080152,tilda,1460,333.0,-1460.0,-1460.0,-1176.0,-1460.0,-527.0,80.0,92.0,...,-1460.0,-492.0,-31.0,-1460.0,-1460.0,-1198.0,-1460.0,-1460.0,-1460.0,-186.0
4074934,dzen,1306,667.0,-156.0,51.0,1306.0,684.0,-425.0,1306.0,654.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2783334,Wikipedia,1428,553.0,-238.0,64.0,92.0,794.0,-52.0,-1448.0,-1448.0,...,-1448.0,-1448.0,-1448.0,-1448.0,-1448.0,-1448.0,-1448.0,-1448.0,-1448.0,-1448.0
5183379,kontur,1416,696.0,-250.0,80.0,92.0,758.0,602.0,-311.0,-84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3423820,apple_push,517,-1448.0,-1448.0,-693.0,80.0,1046.0,-68.0,1448.0,600.0,...,69.0,-24.0,1448.0,600.0,255.0,-26.0,-40.0,40.0,-24.0,1448.0


In [11]:
df.memory_usage(deep=True).sum() / 1024**2

np.float64(2317.443407058716)

Казалось бы по определению длина - целая виличина, а тут используеться float64, давайте экономить память. Проверим диапозон значений, найдем есть ли наны или инфы, проверим количество не целых чисел

In [12]:
target_col = "app_service"
feat_cols = [c for c in df.columns if c.startswith("tcp_len_")]
print(feat_cols)

['tcp_len_1', 'tcp_len_2', 'tcp_len_3', 'tcp_len_4', 'tcp_len_5', 'tcp_len_6', 'tcp_len_7', 'tcp_len_8', 'tcp_len_9', 'tcp_len_10', 'tcp_len_11', 'tcp_len_12', 'tcp_len_13', 'tcp_len_14', 'tcp_len_15', 'tcp_len_16', 'tcp_len_17', 'tcp_len_18', 'tcp_len_19', 'tcp_len_20', 'tcp_len_21', 'tcp_len_22', 'tcp_len_23', 'tcp_len_24', 'tcp_len_25', 'tcp_len_26', 'tcp_len_27', 'tcp_len_28', 'tcp_len_29', 'tcp_len_30']


In [13]:
stats = {
    "min_value": df[feat_cols].min().min(),
    "max_value": df[feat_cols].max().max(),
    "nan_count": np.isnan(df[feat_cols].values).sum(),
    "inf_count": np.isinf(df[feat_cols].values).sum(),
    "non_integer_count": np.sum((df[feat_cols].values % 1) != 0),
    "zero_value": np.sum((df[feat_cols].values == 0)),
}

stats

{'min_value': -1464.0,
 'max_value': 1460.0,
 'nan_count': np.int64(0),
 'inf_count': np.int64(0),
 'non_integer_count': np.int64(0),
 'zero_value': np.int64(75589496)}

Как видно значения по в диапозоне от -1464 до 1460 нет ни нанов ни инфов ни дробных чисел - изменим тип на int16

In [14]:
feat_cols = [c for c in df.columns if c.startswith("tcp_len_")]
df[feat_cols] = df[feat_cols].astype(np.int16)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8248546 entries, 0 to 8248545
Data columns (total 31 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   app_service  object
 1   tcp_len_1    int16 
 2   tcp_len_2    int16 
 3   tcp_len_3    int16 
 4   tcp_len_4    int16 
 5   tcp_len_5    int16 
 6   tcp_len_6    int16 
 7   tcp_len_7    int16 
 8   tcp_len_8    int16 
 9   tcp_len_9    int16 
 10  tcp_len_10   int16 
 11  tcp_len_11   int16 
 12  tcp_len_12   int16 
 13  tcp_len_13   int16 
 14  tcp_len_14   int16 
 15  tcp_len_15   int16 
 16  tcp_len_16   int16 
 17  tcp_len_17   int16 
 18  tcp_len_18   int16 
 19  tcp_len_19   int16 
 20  tcp_len_20   int16 
 21  tcp_len_21   int16 
 22  tcp_len_22   int16 
 23  tcp_len_23   int16 
 24  tcp_len_24   int16 
 25  tcp_len_25   int16 
 26  tcp_len_26   int16 
 27  tcp_len_27   int16 
 28  tcp_len_28   int16 
 29  tcp_len_29   int16 
 30  tcp_len_30   int16 
dtypes: int16(30), object(1)
memory usage: 534.9+ MB


In [15]:
df.memory_usage(deep=True).sum() / 1024**2

np.float64(901.4866428375244)

Почти в 4 раза экономнее, RAM в колабе скажет спасибо

In [16]:
N = len(df)
N

8248546

In [17]:
X0 = df[feat_cols]
y = df[target_col].astype(str)

sample_n = 1_000_000 # всё не влезет
idx = np.random.RandomState(42).choice(N, size=sample_n, replace=False)

X0 = X0.iloc[idx]
y  = y.iloc[idx]

Посмотрим на целевую переменную

In [18]:
y.value_counts()

,count
app_service,
Telegram,12315
287,12277
GMail,12277
Wildberries,12268
Teams,12266
...,...
apteka-ru,135
appdynamics,133
MEGA,131


In [19]:
set(y)

{'1',
 '1-password',
 '101-radio',
 '113',
 '120',
 '15',
 '218',
 '287',
 '299',
 '2gis',
 '343',
 '4',
 '93',
 'Amazon',
 'Anydesk',
 'Apple',
 'Baidu',
 'DataSaver',
 'DoH_DoT',
 'DropBox',
 'Facebook',
 'GMail',
 'Github',
 'Gitlab',
 'Instagram',
 'Ktalk',
 'LinkedIn',
 'Livejournal',
 'MEGA',
 'MSN',
 'Mattermost',
 'Odnoklassniki',
 'Ozon',
 'PlayStore',
 'Playstation',
 'QQ',
 'Quora',
 'Reddit',
 'Rutube',
 'Skype',
 'Snapchat',
 'SoundCloud',
 'Spotify',
 'Steam',
 'Teams',
 'Telegram',
 'TikTok',
 'Twitch',
 'Twitter',
 'WhatsApp',
 'Wikipedia',
 'Wildberries',
 'Yandex',
 'YouTube',
 'adblock-plus',
 'adobe',
 'aeroflot-online',
 'afisha-ru',
 'alfabank',
 'ali-wangwang-file-transfer',
 'alipay',
 'amd-online',
 'andata',
 'any-run',
 'anygo',
 'appdynamics',
 'apple-community',
 'apple-siri',
 'apple_icloud',
 'apple_push',
 'apteka-ru',
 'avito',
 'azure',
 'banki-ru',
 'beeline-online',
 'bing',
 'bitly',
 'blog-posting',
 'braintree',
 'bugsnag',
 'bybit',
 'ca-technolo

### Feature engineering

In [20]:
X0.head()

,tcp_len_1,tcp_len_2,tcp_len_3,tcp_len_4,tcp_len_5,tcp_len_6,tcp_len_7,tcp_len_8,tcp_len_9,tcp_len_10,...,tcp_len_21,tcp_len_22,tcp_len_23,tcp_len_24,tcp_len_25,tcp_len_26,tcp_len_27,tcp_len_28,tcp_len_29,tcp_len_30
3899906,517,-1448,-1448,-1448,-23,64,992,-179,-62,-31,...,586,31,-251,1448,54,31,-305,-31,586,31
5090771,543,-399,51,234,-238,-1460,-1460,-1209,251,-1405,...,-1460,-633,245,-1405,-1460,-1460,-7,249,-1405,-1359
6160203,517,-203,314,-1460,-349,-1136,250,-1391,-1137,250,...,0,0,0,0,0,0,0,0,0,0
3764889,517,-1448,-1448,-579,64,1208,56,-1208,-1448,-946,...,0,0,0,0,0,0,0,0,0,0
7759903,1460,647,-1460,-1460,-1176,-1287,64,92,548,-287,...,0,0,0,0,0,0,0,0,0,0


In [21]:
X = X0.copy()

pos = (X0 > 0)
neg = (X0 < 0)
nonzero = (X0 != 0)
not_zeros = (X0.iloc[:, :-1].values != 0) & (X0.iloc[:, 1:].values != 0)
switches = (pos.iloc[:, :-1].values != pos.iloc[:, 1:].values)
switches_full = np.column_stack([np.zeros(switches.shape[0], dtype=bool), switches])

actual_switches = (switches_full & nonzero)
group_ids = np.cumsum(actual_switches, axis=1)
first_request = np.where(group_ids == 0, X0.values, 0)
first_response = np.where(group_ids == 1, X0.values, 0)
second_request = np.where(group_ids == 2, X0.values, 0)
second_response = np.where(group_ids == 3, X0.values, 0)
third_request = np.where(group_ids == 4, X0.values, 0)
third_response = np.where(group_ids == 5, X0.values, 0)
fourth_request = np.where(group_ids == 6, X0.values, 0)
fourth_response = np.where(group_ids == 7, X0.values, 0)
fifth_request = np.where(group_ids == 8, X0.values, 0)
fifth_response = np.where(group_ids == 9, X0.values, 0)

X["pkt_cnt"] = nonzero.sum(axis=1).astype(np.int16) # общее количество пакетов в потоке
X["n_up"]    = pos.sum(axis=1).astype(np.int16) # количество пакетов от клиента
X["n_down"]  = neg.sum(axis=1).astype(np.int16) # количество пакетов от сервера

X["bytes_up"]   = X0.where(pos, 0).sum(axis=1).astype(np.int32) # суммарное количество байт, отправленных клиентом
X["bytes_down"] = (-X0.where(neg, 0)).sum(axis=1).astype(np.int32) # суммарное количество байт, полученных от сервера
X["total_switches"] = (actual_switches.sum(axis=1)).astype(np.int16) # количество смен направлений
X["first_request"] = np.sum(first_request, axis=1)
X["first_response"] = np.sum(first_response, axis=1)
X["second_request"] = np.sum(second_request, axis=1)
X["second_response"] = np.sum(second_response, axis=1)
X["third_request"] = np.sum(third_request, axis=1)
X["third_response"] = np.sum(third_response, axis=1)
X["fourth_request"] = np.sum(fourth_request, axis=1)
X["fourth_response"] = np.sum(fourth_response, axis=1)
X["fifth_request"] = np.sum(fifth_request, axis=1)
X["fifth_response"] = np.sum(fifth_response, axis=1)


print(X["total_switches"].min().min())
print(X["total_switches"].max().max())
print(X["total_switches"].mode()[0])


print(X.iloc[0])
print(switches_full[0])

0
29
5
tcp_len_1           517
tcp_len_2         -1448
tcp_len_3         -1448
tcp_len_4         -1448
tcp_len_5           -23
tcp_len_6            64
tcp_len_7           992
tcp_len_8          -179
tcp_len_9           -62
tcp_len_10          -31
tcp_len_11          -35
tcp_len_12           31
tcp_len_13         -251
tcp_len_14           31
tcp_len_15         1448
tcp_len_16          162
tcp_len_17          -35
tcp_len_18           31
tcp_len_19         -305
tcp_len_20          -31
tcp_len_21          586
tcp_len_22           31
tcp_len_23         -251
tcp_len_24         1448
tcp_len_25           54
tcp_len_26           31
tcp_len_27         -305
tcp_len_28          -31
tcp_len_29          586
tcp_len_30           31
pkt_cnt              30
n_up                 15
n_down               15
bytes_up           6043
bytes_down         5883
total_switches       14
first_request       517
first_response    -4367
second_request     1056
second_response    -307
third_request        31
third_res

In [22]:
def analyze_tcp_flows(matrix):
    results = []

    for row in matrix:
        # Убираем нулевые пакеты (паддинг), если они есть
        packets = row[row != 0]
        if len(packets) == 0:
            results.append({"requests": 0, "responses": 0, "first_3_req": [], "first_3_res": []})
            continue

        # Определяем смену направления (когда знак текущего пакета не равен знаку предыдущего)
        # signs: True для клиента (>0), False для сервера (<0)
        signs = packets > 0
        switch_direction = np.insert(signs[1:] != signs[:-1], 0, True)

        # Группируем пакеты по фазам (индекс группы инкрементируется при смене знака)
        group_ids = np.cumsum(switch_direction)

        # Считаем суммарный размер каждого сообщения
        messages = []
        for g_id in np.unique(group_ids):
            msg_sum = np.sum(packets[group_ids == g_id])
            messages.append(msg_sum)

        # Разделяем на реквесты (положительные) и респонсы (отрицательные)
        requests = [m for m in messages if m > 0]
        responses = [abs(m) for m in messages if m < 0]

        results.append({
            "total_requests": len(requests),
            "total_responses": len(responses),
            "first_3_requests_sizes": requests[:3],
            "first_3_responses_sizes": responses[:3]
        })

    return results

# Пример использования:
# 100, 50 (один реквест) | -200, -300 (один респонс) | 150 (второй реквест)
matrix = np.array([
    [100, 50, -200, -300, 150, -10, -20, 0, 0, 0] # и так далее до 30
])

analysis = analyze_tcp_flows(matrix)
print(analysis[0])

{'total_requests': 2, 'total_responses': 2, 'first_3_requests_sizes': [np.int64(150), np.int64(150)], 'first_3_responses_sizes': [np.int64(500), np.int64(30)]}


### Обучение модели и валидация

In [23]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
model = CatBoostClassifier(
    loss_function="MultiClass",
    eval_metric="TotalF1",
    auto_class_weights='Balanced',
    # custom_metric=["Precision", "Recall", "Accuracy"],
    iterations=1000,
    depth=10,
    learning_rate=0.15,
    random_seed=42,
    verbose=True,
    task_type="GPU",
    devices="0"
)

model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    use_best_model=True
)

pred = model.predict(X_val).reshape(-1)
print("val accuracy:", accuracy_score(y_val, pred))

0:	learn: 0.2249258	test: 0.2169201	best: 0.2169201 (0)	total: 9.65s	remaining: 2h 40m 40s
1:	learn: 0.2480010	test: 0.2431540	best: 0.2431540 (1)	total: 20.2s	remaining: 2h 48m 5s
2:	learn: 0.2714840	test: 0.2651989	best: 0.2651989 (2)	total: 29.3s	remaining: 2h 42m 30s
3:	learn: 0.3422502	test: 0.3357996	best: 0.3357996 (3)	total: 38.7s	remaining: 2h 40m 43s
4:	learn: 0.3624045	test: 0.3589908	best: 0.3589908 (4)	total: 49.1s	remaining: 2h 42m 42s
5:	learn: 0.3813159	test: 0.3766863	best: 0.3766863 (5)	total: 58.4s	remaining: 2h 41m 22s
6:	learn: 0.4117643	test: 0.4043336	best: 0.4043336 (6)	total: 1m 7s	remaining: 2h 40m 32s
7:	learn: 0.4441605	test: 0.4333640	best: 0.4333640 (7)	total: 1m 17s	remaining: 2h 40m 58s
8:	learn: 0.4721334	test: 0.4609169	best: 0.4609169 (8)	total: 1m 27s	remaining: 2h 40m 26s
9:	learn: 0.4967259	test: 0.4846353	best: 0.4846353 (9)	total: 1m 37s	remaining: 2h 40m 21s
10:	learn: 0.5268994	test: 0.5093865	best: 0.5093865 (10)	total: 1m 48s	remaining: 2h 42

In [26]:
from sklearn.metrics import classification_report
print(classification_report(y_val, pred))

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                                   precision    recall  f1-score   support

                                1       0.22      0.22      0.22      2414
                       1-password       0.00      0.00      0.00        62
                        101-radio       0.00      0.00      0.00        27
                              113       0.37      0.75      0.50      2416
                              120       0.83      0.72      0.77      2417
                               15       0.18      0.14      0.16       538
                              218       0.52      0.30      0.38        82
                              287       0.79      0.96      0.87      2455
                              299       0.66      0.38      0.49      2453
                             2gis       0.62      0.57      0.60      1821
                              343       0.34      0.88      0.49      2437
                                4       0.75      0.89      0.81      2426
                        

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# <font color="green">Внедрение и deployment </font>


### Онлайн vs оффлайн

- Онлайн: классификация каждого TCP-потока на лету → низкая задержка, нужна оптимизация модели.

- Оффлайн: анализ собранных логов → более тяжёлые модели, batch processing.


### Архитектура

- Data collector → потоковые данные о TCP → Preprocessing → Model → Results

- Возможна интеграция с корпоративным мониторингом.


### Выход модели

- Прямое приложение/сервис

- Top-K вероятности для учета неопределённости

- Возможность отбрасывать низкоуверенные предсказания.

# <font color="green">Логгирование и мониторинг </font>


### Что логировать

- Входные потоки (только метаданные, без payload)

- Предсказанные классы и вероятности

- Ошибки классификации (если есть метки)

- Время обработки


### Мониторинг

- Performance drift: точность со временем, особенно при обновлениях приложений

- Data drift: изменение распределения длины пакетов и направлений

- Модельный лог: уверенность предсказания, частота каждого класса

- Alerts: на резкие изменения трафика или появление неизвестных приложений


### Метрики производительности

- Latency per flow

- Throughput (flows/sec)

- Resource usage (CPU, RAM)